In [ ]:
import pickle
import glob
import os
from tqdm import tqdm
import numpy as np
import os

current_directory = os.getcwd()
if 'experiments' in current_directory:
    parent_directory = os.path.dirname(current_directory)
    os.chdir(parent_directory)

ds = 'cityscapes'
num_classes = 19
exp = 'best_thresh'
exp_dir = f'log/{ds}/{exp}'


overall_dict = {}

for file in tqdm(glob.glob(os.path.join(exp_dir, '*.pkl')), desc=f'reading from {exp_dir}'):
    d = pickle.load(open(file, 'rb'))
    filename = os.path.basename(file).replace('.pkl', '')
    new_d = {}
    new_d[filename] = d[filename]
    d = new_d
    for image_name, image_d in d.items():
        for model_type, model_d in image_d.items():
            if model_type not in overall_dict:
                overall_dict[model_type] = {}
            for metric, value in model_d.items():
                if isinstance(value, np.ndarray):
                    value = sum(value)
                if isinstance(value, dict):
                    for k, v in value.items():
                        if k == 'num_pixels': k='num_pixels_per_cls'
                        if k == 'cig': k='cig_per_cls'
                        if k not in overall_dict[model_type]:
                            overall_dict[model_type][k] = v
                        else:
                            overall_dict[model_type][k] += v
                    continue
                if metric not in overall_dict[model_type]:
                    overall_dict[model_type][metric] = value
                else:
                    overall_dict[model_type][metric] += value


In [ ]:
# graph_d should contain the relevant calculations
# graph_d = {
graph_d = {}
baseline = {}
for model_type, model_d in overall_dict.items():

    if isinstance(model_type, tuple):
        if model_type not in graph_d:
            graph_d[model_type] = {}

        assert sum(model_d['num_pixels_per_cls']) == model_d['num_pixels']
        graph_d[model_type]['CIG'] = sum(model_d['cig_per_cls'])/sum(model_d['num_pixels_per_cls'])/np.log(171)
        graph_d[model_type]['cCIG'] = np.mean([c/n/np.log(171) for c, n in zip(model_d['cig_per_cls'], model_d['num_pixels_per_cls']) if n > 0])
        graph_d[model_type]['%\\ certified'] = model_d['certified_count']/model_d['num_pixels']
    else:
        baseline[model_type] = {}
        baseline[model_type]['confusion_matrix'] = model_d['confusion_matrix']


In [ ]:
graph_d = dict(sorted(graph_d.items(), key=lambda item: item[1]['CIG'], reverse=True))
top_k = 0; d = False
for model_type, model_d in graph_d.items():
    n, n0, f, h_i, sigma, tau = model_type
    if f is None:
        print('\nSegCertify', model_type, model_d)
        d = True
    else:
        if top_k < 10:
            print('AdaptiveCertify', model_type, model_d)
            top_k +=1
    if top_k == 10 and d:
        break
